## データの読込

In [1]:
import pandas as pd
df = pd.read_csv("vgsales-12-4-2019.csv")
df.head(3)

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,...,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,...,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,...,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,...,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg


In [2]:
import os
import time
import random
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from natsort import natsorted 

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import warnings
warnings.simplefilter('ignore')

In [3]:
# シード値の固定
SEED = 42

random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
                                                    df.dropna(subset=["Global_Sales"]).drop(["Global_Sales",  "NA_Sales", "PAL_Sales", "JP_Sales", "Other_Sales"], axis=1), 
                                                    df.dropna(subset=["Global_Sales"])["Global_Sales"],  
                                                    test_size=0.3,
                                                    shuffle=True, 
                                                    random_state=SEED
                                                    ) 

## パイプライン

In [5]:
# 数値データカラム名を取得
number_columns = df.drop(["Global_Sales",  "NA_Sales", "PAL_Sales", "JP_Sales", "Other_Sales"], axis=1).select_dtypes(include="number").columns

# カテゴリデータカラム名を取得
category_columns = df.drop(["Global_Sales",  "NA_Sales", "PAL_Sales", "JP_Sales", "Other_Sales"], axis=1).select_dtypes(include="object").columns

In [6]:
# カテゴリデータカラムの各カラムのカテゴリーの数を取得
category_unique_num = df.select_dtypes(include="object").nunique()

In [7]:
# 数値データ用の変換
numeric_transformer = Pipeline(steps=[
    ('num_imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())]
)

In [8]:
# カテゴリーの数にしきい値を設けて、カテゴリー数の多いカラムと少ないカラムに分ける
thread = 10

many_kinds_category_columns = category_unique_num[category_unique_num >= thread].index
few_kinds_category_columns = category_unique_num[category_unique_num < thread].index

In [9]:
# カテゴリーのエンコーディング法則を指定する
ordinal_all_cols_mapping = []

for column in many_kinds_category_columns:
    ordinal_one_cols_mapping = []
    for category in natsorted(X_train[column].unique()):
        ordinal_one_cols_mapping.append(category)

    ordinal_all_cols_mapping.append(ordinal_one_cols_mapping)

In [10]:
# カテゴリー数が多いカテゴリーデータ用の変換
many_kinds_categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal', OrdinalEncoder(
                handle_unknown = 'use_encoded_value', # 未知数をunknown valueに置き換える設定
                unknown_value = -1,
                categories = ordinal_all_cols_mapping
            )
        )
])

In [11]:
# カテゴリー数が少ないカテゴリーデータ用の変換
few_kinds_categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

In [12]:
# ColumnTransformerの作成
columns_transformers = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, number_columns),
        ('many_kinds', many_kinds_categorical_transformer, many_kinds_category_columns),
        ('few_kinds', few_kinds_categorical_transformer, few_kinds_category_columns)
    ]
)

In [13]:
# パイプライン全体の設定
step = [
        ("columns_transformers", columns_transformers),
        ('model', lgb.LGBMRegressor(random_state=42))
     ]

In [14]:
# パイプラインの作成
pipe = Pipeline(
    step
)

In [15]:
# 学習
pipe.fit(X_train, y_train)

Pipeline(steps=[('columns_transformers',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('num_imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Rank', 'VGChartz_Score', 'Critic_Score', 'User_Score', 'Total_Shipped',
       'Year', 'status', 'Vgchartzscore'],
      dtype='object')),
                                                 ('many_kinds',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImpu...
                                                                                  unknown_value=-1))]),
                             

In [16]:
# 推論
y_pred = pipe.predict(X_test)

### GroupKFold を併用

In [17]:
# 学習・推論
gkf = GroupKFold(n_splits=5)

groups = X_train["Genre"]

best_params, history = {}, []

cv_result = []

for i, (train_index, test_index) in enumerate(gkf.split(X_train, y_train, groups)):
    X_train_gkf, X_test_gkf = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_gkf, y_test_gkf = y_train.iloc[train_index], y_train.iloc[test_index]

    # 学習、推論
    pipe.fit(X_train_gkf, y_train_gkf)

    y_pred = pipe.predict(X_test_gkf)

    rmse = mean_squared_error(y_test_gkf, y_pred, squared=False)
    cv_result.append(rmse)

print("RMSE:", cv_result)
print("RMSE:", np.mean(cv_result))

RMSE: [0.3381069648542431, 0.10717670046871884, 0.15152424264212594, 0.02716385550389459, 0.3365058004617997]
RMSE: 0.1920955127861564


### Stackingを実施

In [18]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from catboost import CatBoostRegressor

In [19]:
models = {
    "ridge":Ridge(random_state=SEED),
    "lasso":Lasso(random_state=SEED),
    "linear":LinearRegression(),
    "elastic_net":ElasticNet(random_state=SEED),
    "svm":SVR(),
    "random_forest":RandomForestRegressor(random_state=SEED),
    "gradient":GradientBoostingRegressor(random_state=SEED),
    "catboost":CatBoostRegressor(random_state=SEED, 
                                 silent=True, # ログを非表示
                                 ),
    "xgboost":xgb.XGBRegressor(
        random_state=SEED,
        objective='reg:squarederror'
        ),
    "lightgbm":lgb.LGBMRegressor(random_state=SEED),
}

### 一層目

In [20]:
gkf = GroupKFold(n_splits=5)
groups = X_train["Genre"]

cv_result_stck = {}
pred_df = pd.DataFrame()

for i, (model_name, model) in enumerate(models.items()):

    print(i, model)

    # パイプライン全体の設定
    step = [
        ("columns_transformers", columns_transformers),
        ('model', model)
     ]

    # パイプラインの作成
    pipe = Pipeline(
        step
    )

    each_model_df = pd.DataFrame()
    each_model_result = []

    for train_index, test_index in gkf.split(X_train, y_train, groups):

        X_train_gkf, X_test_gkf = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_gkf, y_test_gkf = y_train.iloc[train_index], y_train.iloc[test_index]

        pipe.fit(X_train_gkf, y_train_gkf)
        y_pred = pipe.predict(X_test_gkf)

        tmp_df = pd.DataFrame(
                        [y_pred],
                        columns=test_index
                    )

        rmse = mean_squared_error(y_test_gkf, y_pred, squared=False)
        each_model_result.append(rmse)
        each_model_df = pd.concat([each_model_df , tmp_df.T]) # 各KFold ごとの予測結果をDataFrameに縦に並べる

    cv_result_stck[model_name] = each_model_result # 各モデルのRMSEを集計
    each_model_df.columns = [model_name] # カラム名をモデル名に変更
    pred_df = pd.concat([pred_df, each_model_df.sort_index()], axis=1) # 予測結果集計用DataFrameに各モデルの予測結果をくっつける

0 Ridge(random_state=42)
1 Lasso(random_state=42)
2 LinearRegression()
3 ElasticNet(random_state=42)
4 SVR()
5 RandomForestRegressor(random_state=42)
6 GradientBoostingRegressor(random_state=42)
7 <catboost.core.CatBoostRegressor object at 0x0000023AE3159488>
8 XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None,
             enable_categorical=False, gamma=None, gpu_id=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=None)
9 LGBMRegressor(random_state=42)


In [21]:
print(len(pred_df))

pred_df.head()

13590


,ridge,lasso,linear,elastic_net,svm,random_forest,gradient,catboost,xgboost,lightgbm
0,0.443119,0.449006,0.443135,0.392393,0.169451,0.16,0.164407,0.162476,0.159579,0.159485
1,0.845699,0.486278,0.845692,0.515446,0.163745,0.56,0.547240,0.557286,0.559839,0.556271
2,0.524295,0.308543,0.524336,0.265163,0.129279,0.22,0.219579,0.214526,0.219991,0.219870
3,1.222256,0.385726,1.222277,0.399713,0.135002,0.84,0.824000,0.837600,0.839275,0.839208
4,0.702510,0.390081,0.702716,0.394165,0.272038,0.23,0.237628,0.236664,0.230394,0.229865


In [22]:
best_model_score = 9999
best_model_name = ""

In [23]:
for model_name, rmse in cv_result_stck.items():
    print(model_name, np.mean(rmse))

    if best_model_score > np.mean(rmse):
        best_model_score = np.mean(rmse)
        best_model_name = model_name

print()
print("Best Model is ", best_model_name, "! Best Score is ", best_model_score, "!")

ridge 0.6373001236909286
lasso 0.7939934392397442
linear 0.6373005953395999
elastic_net 0.7918085980789809
svm 0.8138065516676185
random_forest 0.04890771176895532
gradient 0.03150057457256164
catboost 0.2199297740079543
xgboost 0.03488328732035569
lightgbm 0.1920955127861564

Best Model is  gradient ! Best Score is  0.03150057457256164 !


### 2階層目 各モデルの予測結果をもとに予測

In [24]:
# 学習・推論
gkf = GroupKFold(n_splits=5)

y_train
y_test

groups = pred_df[best_model_name] # 最もスコアの良かった model_name をGroup に設定

cv_result_stck = []

step = [
    ("numeric_transformers", numeric_transformer), # 全て数値カラムなので、数値カラムのTransformerに変更
    ('model', models[best_model_name]) # 最もスコアの良かったモデルを使用
]

# パイプラインの作成
pipe = Pipeline(
    step
)

for train_index, test_index in gkf.split(pred_df, y_train, groups):

    X_train_gkf, X_test_gkf = pred_df.iloc[train_index], pred_df.iloc[test_index]
    y_train_gkf, y_test_gkf = y_train.reset_index(drop=True).iloc[train_index], y_train.reset_index(drop=True).iloc[test_index]

    pipe.fit(X_train_gkf, y_train_gkf)
    y_pred = pipe.predict(X_test_gkf)

    rmse = mean_squared_error(y_test_gkf, y_pred, squared=False)
    cv_result_stck.append(rmse)

print("RMSE:", cv_result_stck)
print("RMSE:", np.mean(cv_result_stck))

RMSE: [0.02193021869744928, 0.02361002538127753, 0.024184080353046512, 0.11497196105629219, 0.08212197431851904]
RMSE: 0.053363651961316906


In [25]:
print("RMSE:", np.mean(cv_result))
print("Stacking RMSE:", np.mean(cv_result_stck))

RMSE: 0.1920955127861564
Stacking RMSE: 0.053363651961316906
